<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preambles" data-toc-modified-id="Preambles-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preambles</a></span></li><li><span><a href="#System-parameters" data-toc-modified-id="System-parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>System parameters</a></span></li><li><span><a href="#Gate-parameters" data-toc-modified-id="Gate-parameters-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Gate parameters</a></span></li><li><span><a href="#Optimizing-fidelity-by-changing-the-pulse's-detuning-and-drag-coeff" data-toc-modified-id="Optimizing-fidelity-by-changing-the-pulse's-detuning-and-drag-coeff-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Optimizing fidelity by changing the pulse's detuning and drag coeff</a></span></li><li><span><a href="#Sweep-gate-time-(optional)" data-toc-modified-id="Sweep-gate-time-(optional)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Sweep gate time (optional)</a></span></li><li><span><a href="#Pauli-Transfer-Matrix" data-toc-modified-id="Pauli-Transfer-Matrix-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Pauli Transfer Matrix</a></span></li><li><span><a href="#Estimating-error-channel" data-toc-modified-id="Estimating-error-channel-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Estimating error channel</a></span><ul class="toc-item"><li><span><a href="#Ideal-PTM" data-toc-modified-id="Ideal-PTM-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Ideal PTM</a></span></li><li><span><a href="#PTM-to-$\chi$" data-toc-modified-id="PTM-to-$\chi$-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>PTM to $\chi$</a></span></li><li><span><a href="#PTM-to-operator" data-toc-modified-id="PTM-to-operator-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>PTM to operator</a></span></li><li><span><a href="#$\chi$-to-unitary-operator" data-toc-modified-id="$\chi$-to-unitary-operator-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>$\chi$ to unitary operator</a></span></li></ul></li></ul></div>

## The Plan

*Would be great if qchard could be upgraded to modern python and QuTip*


Simulations to do (likely in this order):
1. IdealGridium-Transmon, charge coupling (easiest thing to simulate)
2. IdealGridium-Fluxonium (tunable)
3. ExpGridium-Transmon
4. ExpGridium-Fluxonium

Noteable values to extract:
- Extract the energy shifts as you turn on coupling
- Apply the drive via master equation solver.
- Maximum population transfer to a ideal C(theta) gate

Things to think about:
- Readout is coupled to a particular mode. This mode is what we diagonalize on and have some dispersive shift for. However, in the real world, there are other couplings which make the real projected values not as straightforward. Would be interesting at some point to investigate this more thoroughly.

### Holdups

To upgrade the versioning (which is highly desired due to the increased stability and increase solve rate) I am mainly running into an issue surrounding the handling of the * operator between lists/numpy arrays and Qobj object.

# Preambles

In [ ]:
import sys
sys.path.append('Users/thomasersevim/anaconda3')
sys.path.append('/Users/thomasersevim/QNL/2q_gridium/')
sys.dont_write_bytecode = True
import time
import numpy as np
from matplotlib import pyplot as plt
import scipy.integrate
from scipy.optimize import minimize
from qutip import *
import plotting_settings
import matplotlib as mpl
from collections import OrderedDict
mpl.rcParams['figure.dpi']= 400
from itertools import product
import scqubits
plt.close('all')

from Circuit_Objs.qchard_idealgridium import *
from Circuit_Objs.qchard_transmon import *
from Circuit_Objs import qchard_coupobj as coupobj
from Circuit_Objs import qchard_evolgates as gates

## 1. IdealGridium-Transmon

In [ ]:
# Better way of doing all this
cphase_resonant_transition = 3
GSQ = IdealGridium(**soft_IdealGridium_params, **std_IdealGridium_sim_params)
transmon01_freq = GSQ.transition_energies()[cphase_resonant_transition+1]
transmon = transmon_creation_from_01(linear_freq=transmon01_freq)

J_C = 0.01 # Strength of the capacitive coupling between the two
T_gate = 50 # An integer that factors into total number of time steps in propegator (2*T_gate+1 steps)

# Pulse shape
shape = 'cos'  # 'gauss', 'cos' for 1-cos, or 'square'
sigma = 0.25  # sigma in units of T_gate for shape=='gauss'
drag = False
drag_coeff = 0.2

# Scaling of the ideal value given by the inverse matrix element.
drive_amplitude_factor = 1  # 0.95436

# Drive frequency with respect to the resonance.
delta_omega_d = 0

# Method to calculate the propagator.
# 'propagator - full propagator using qt.propagator
# 'sesolve' - propagator using qt.sesolve for 4 computational states
method = 'propagator'

# Hilbert space (of what?)
# nlev_cav = 4
nlev_q = 5

save_figure = False
# filename_prefix = 'stuff'

# Indices of the computational space.
comp_space = ['00', '01', '10', '11']
interaction = 'on'

# Defining "left" vs "right" qubit
qubitA = transmon
qubitB = GSQ

transition_to_drive = ('00', '03')
# transition_to_drive = ('10', '13')

# Transitions to show
states011 = ['13', '12'] # Both promoted
states010 = ['10', '00'] # Only A promoted
states001 = ['02', '03'] # Only B promoted
states000 = ['00'] # Neither promoted

# Couple the two qubits
system = coupobj.CoupledObjects(qubitA, qubitB, [qubitA, qubitB, J_C, 'charge'])

# Actual Procedure
For clarity, we want a drivable transition of the gridium to correspond to the transition frequency of an adjacent transmon. So, for example, let's say the 0->3 transition is strong through a capacitive drive. This splitting should be mached to the transmon 0-1 splitting. Now the 0-3 transition will break degeneracy with a state dependence on the transmon 0-1 state. This means driving the gridium 0-3 transition (directly between the 00-03 and 10-13 transition |transmon gridium>) gives a transmon state dependent phase on the gridium, aka a controlled phase (CPHASE).

# Gate parameters

In [ ]:
# Calculate the drive frequency.
level1, level2 = transition_to_drive[0], transition_to_drive[1]
omega_d = abs(system.freq(level1, level2)) + delta_omega_d

# Calculate the drive amplitude.
matr_el = np.abs(system.n_ij(qubitA, level1, level2) + system.n_ij(qubitB, level1, level2))
epsilon = drive_amplitude_factor / abs(matr_el)

# Printing relevant parameters
print('Detuning between 00-03 and 10-13: {:.1f} MHz'.format(1000*np.abs(system.freq('00', '03') - system.freq('10', '13'))))
print('Transition to drive: {} - {} with frequency {:.4f} GHz'.format(level1, level2, abs(system.freq(level1, level2))))
print('Drive frequency: {:.4f} GHz'.format(omega_d))
print('Drive amplitude scale factor: {:.4f}'.format( drive_amplitude_factor))

In [ ]:
t_points = np.linspace(0, T_gate, 2 * int(T_gate) + 1)

# The time-independent operator part of the drive term.
H_drive = epsilon * (system.n(0) + system.n(1))
H_drive_dummy = 0 * (system.n(0) + system.n(1))
if method == 'sesolve':
    # This calculates the evolution operator that works for
    # computational levels only.
    U_t = gates.evolution_compspace_microwave(system.H(), H_drive, comp_space=comp_space, t_points=t_points,T_gate=T_gate, shape=shape, sigma=sigma, omega_d=omega_d,interaction=interaction)

elif method == 'propagator':
    # This calculates the evolution operator for the whole system  
    U_t = gates.evolution_operator_microwave(system.H(), H_drive, comp_space=comp_space, t_points=t_points,T_gate=T_gate, shape=shape, sigma=sigma, DRAG = drag, DRAG_coefficient = drag_coeff, omega_d=omega_d,interaction=interaction)
    U_nothing = gates.evolution_operator_microwave(-system.H(), H_drive_dummy, comp_space=comp_space, t_points=t_points,T_gate=T_gate, shape=shape, sigma=sigma, DRAG = drag, DRAG_coefficient = drag_coeff, omega_d=omega_d,interaction=interaction)

In [ ]:
U_real = gates.change_operator_proj_subspace(
        system, U_t, subspace=comp_space, interaction=interaction)
single_qubit_gates = gates.operator_single_qub_z(system, U_real[-1])
fidelity = gates.fidelity_cz_gate(
    system, U_t, comp_space=comp_space,
    interaction=interaction, single_gates='z')
#Note: this is only for unitary evolution. We shall investigate dephasing errors later.
print('max fidelity during the simulations: ', np.max(fidelity))

print('\n** Final values **')
print('Fidelity: ', fidelity[-1])
print('Diagonal elements of the evolution operator '
      + '(amplitudes and phases with respect to E*t in units of pi)')

In [ ]:
U_f = U_t[-1]
U_me = {}
for state in comp_space:
    vec = system.eigvec(state, interaction=interaction)
    U_me[state] = U_f.matrix_element(vec.dag(), vec)
for state in comp_space:
    print(state, np.abs(U_me[state]),
          (np.angle(U_me[state]
                    * np.exp(2j * np.pi * system.level(state) * T_gate))) / np.pi)
print('$(\phi_{00} + \phi_{11} - \phi_{01} - \phi_{01})/\pi=$')
print((np.angle(U_me['00']) + np.angle(U_me['11'])
       - np.angle(U_me['01']) - np.angle(U_me['10'])) / np.pi)
phase_accum = (np.angle(U_me['00']) + np.angle(U_me['11'])
               - np.angle(U_me['01']) - np.angle(U_me['10']))
phase_accum = phase_accum / np.pi

initial_state = system.eigvec(transition_to_drive[0])
final_state = system.eigvec(transition_to_drive[1])
P_driven_transition = gates.prob_transition(U_t, initial_state, final_state)

t_2nd_excited = scipy.integrate.trapz(P_driven_transition, t_points)
print('Time spent in the 2nd state for {} - {}: {:.1f} ns'.format(
    transition_to_drive[0], transition_to_drive[1],
    t_2nd_excited))

In [ ]:
P011 = {}
P010 = {}
P001 = {}
P000 = {}

fig, axes = plt.subplots(2, 2, figsize=(12, 12))

ax011 = axes[0, 0]
ax010 = axes[0, 1]
ax001 = axes[1, 0]
ax000 = axes[1, 1]

for state in states011:
    P011[state] = gates.prob_transition(U_t, system.eigvec('11'), system.eigvec(state))
    ax011.plot(t_points, P011[state], lw=2,
               label=r'$P(11\rightarrow{})$'.format(state))

for state in states010:
    P010[state] = gates.prob_transition(U_t, system.eigvec('10'), system.eigvec(state))
    ax010.plot(t_points, P010[state], lw=2,
               label=r'$P(10\rightarrow {})$'.format(state))

for state in states001:
    P001[state] = gates.prob_transition(U_t, system.eigvec('01'), system.eigvec(state))
    ax001.plot(t_points, P001[state], lw=2,
               label=r'$P(01\rightarrow {})$'.format(state))

for state in states000:
    P000[state] = gates.prob_transition(U_t, system.eigvec('00'), system.eigvec(state))
    ax000.plot(t_points, P000[state], lw=2,
               label=r'$P(00\rightarrow {})$'.format(state))

textfontsize = 18
fig.text(0.5, 0.3, r'At $t = {}$ ns: '.format(int(t_points[-1])),
        fontsize=textfontsize, ha='center')
fig.text(0.5, 0.25,
        r'$P(11\rightarrow 11) = {:.4f}$, '.format(P011['11'][-1])
        + r'$P(10\rightarrow 10) = {:.4f}$, '.format(P010['10'][-1])
        + r'$P(01\rightarrow 10) = {:.4f}$, '.format(P001['01'][-1])
        + r'$P(00\rightarrow 00) = {:.4f}$'.format(P000['00'][-1]),
        fontsize=textfontsize, ha='center')
ax011.text(0.98, 0.93,
           r'$P(11 \rightarrow 11) = {:.6f}$'.format(P011['11'][-1]),
           ha='right', va='top', transform=ax011.transAxes,
           fontsize=textfontsize)
ax010.text(0.98, 0.93,
           r'$P(10 \rightarrow 10) = {:.6f}$'.format(P010['10'][-1]),
           ha='right', va='top', transform=ax010.transAxes,
           fontsize=textfontsize)
ax001.text(0.98, 0.93,
           r'$P(01 \rightarrow 01) = {:.6f}$'.format(P001['01'][-1]),
           ha='right', va='top', transform=ax001.transAxes,
           fontsize=textfontsize)
ax000.text(0.98, 0.93,
           r'$P(00 \rightarrow 00) = {:.6f}$'.format(P000['00'][-1]),
           ha='right', va='top', transform=ax000.transAxes,
           fontsize=textfontsize)
# fig.text(0.65, 0.25, r'$P(10) = {:.4f}$'.format(
#          P010['10'][-1]), fontsize=textfontsize)
fig.text(0.5, 0.1,
         r'CZ gate phase accumulation: '
         + '$\phi_{00} + \phi_{11} - \phi_{10} - \phi_{01} = $'
         + r'${:.3f} \pi $'.format(phase_accum),
         fontsize=textfontsize, ha='center')
fig.text(0.5, 0.05,
         r'Fidelity: '
         + r'$F = {:.6f}$'.format(fidelity[-1]),
         fontsize=textfontsize, ha='center')

for axarr in axes:
    for ax in axarr:
        ax.legend(loc='lower left')
        ax.set_xlim([np.min(t_points), np.max(t_points)])
        ax.set_xlabel('Time (ns)')
        ax.set_ylim([0, 1.02])
        ax.set_ylabel(r'$P_{i\rightarrow f}$')

ax011.set_title(
    r'Starting in $|11\rangle$')
ax010.set_title(
    r'Starting in $|10\rangle$')

ax001.set_title(
    r'Starting in $|01\rangle$')
ax000.set_title(
    r'Starting in $|00\rangle$')

fig.tight_layout(rect=[0, 0.15, 1, 1])

# Optimizing fidelity by changing the pulse's detuning and drag coeff

In [ ]:
drag = True

def infidelity(x):
    delta_omega_d, drag_coeff = x
    omega_d = abs(system.freq(level1, level2)) + delta_omega_d
    U_t = gates.evolution_operator_microwave(system.H(), H_drive, comp_space=comp_space, t_points=t_points,T_gate=T_gate, shape=shape, sigma=sigma, DRAG = drag, DRAG_coefficient = drag_coeff, omega_d=omega_d,interaction=interaction)
    U_f = U_t[-1]
    return 1-gates.fidelity_cz_gate(system, U_f, comp_space=comp_space,interaction=interaction)

x0 = [0,0]
xopt = minimize(infidelity, x0, method ='Nelder-Mead')
print (xopt.x)
print(1-infidelity(xopt.x))

In [ ]:
delta_omega_d, drag_coeff = xopt.x
omega_d = abs(system.freq(level1, level2)) + delta_omega_d
U_t = gates.evolution_operator_microwave(system.H(), H_drive, comp_space=comp_space, t_points=t_points,T_gate=T_gate, shape=shape, sigma=sigma, DRAG = drag, DRAG_coefficient = drag_coeff, omega_d=omega_d,interaction=interaction)
U_real = gates.change_operator_proj_subspace(
        system, U_t, subspace=comp_space, interaction=interaction)
single_qubit_gates = gates.operator_single_qub_z(system, U_real[-1])
fidelity = gates.fidelity_cz_gate(
    system, U_t, comp_space=comp_space,
    interaction=interaction, single_gates='z')

#Note: this is only for unitary evolution. We shall investigate dephasing errors later.
print('max fidelity during the simulations: ', np.max(fidelity))

print('\n** Final values **')
print('Fidelity: ', fidelity[-1])
print('Diagonal elements of the evolution operator '
      + '(amplitudes and phases with respect to E*t in units of pi)')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

ax011 = axes[0, 0]
ax010 = axes[0, 1]
ax001 = axes[1, 0]
ax000 = axes[1, 1]

for state in states011:
    P011[state] = gates.prob_transition(U_t, system.eigvec('11'), system.eigvec(state))
    ax011.plot(t_points, P011[state], lw=2,
               label=r'$P(11\rightarrow{})$'.format(state))

for state in states010:
    P010[state] = gates.prob_transition(U_t, system.eigvec('10'), system.eigvec(state))
    ax010.plot(t_points, P010[state], lw=2,
               label=r'$P(10\rightarrow {})$'.format(state))

for state in states001:
    P001[state] = gates.prob_transition(U_t, system.eigvec('01'), system.eigvec(state))
    ax001.plot(t_points, P001[state], lw=2,
               label=r'$P(01\rightarrow {})$'.format(state))

for state in states000:
    P000[state] = gates.prob_transition(U_t, system.eigvec('00'), system.eigvec(state))
    ax000.plot(t_points, P000[state], lw=2,
               label=r'$P(00\rightarrow {})$'.format(state))

textfontsize = 18
fig.text(0.5, 0.3, r'At $t = {}$ ns: '.format(int(t_points[-1])),
        fontsize=textfontsize, ha='center')
fig.text(0.5, 0.25,
        r'$P(11\rightarrow 11) = {:.4f}$, '.format(P011['11'][-1])
        + r'$P(10\rightarrow 10) = {:.4f}$, '.format(P010['10'][-1])
        + r'$P(01\rightarrow 10) = {:.4f}$, '.format(P001['01'][-1])
        + r'$P(00\rightarrow 00) = {:.4f}$'.format(P000['00'][-1]),
        fontsize=textfontsize, ha='center')
ax011.text(0.98, 0.93,
           r'$P(11 \rightarrow 11) = {:.6f}$'.format(P011['11'][-1]),
           ha='right', va='top', transform=ax011.transAxes,
           fontsize=textfontsize)
ax010.text(0.98, 0.93,
           r'$P(10 \rightarrow 10) = {:.6f}$'.format(P010['10'][-1]),
           ha='right', va='top', transform=ax010.transAxes,
           fontsize=textfontsize)
ax001.text(0.98, 0.93,
           r'$P(01 \rightarrow 01) = {:.6f}$'.format(P001['01'][-1]),
           ha='right', va='top', transform=ax001.transAxes,
           fontsize=textfontsize)
ax000.text(0.98, 0.93,
           r'$P(00 \rightarrow 00) = {:.6f}$'.format(P000['00'][-1]),
           ha='right', va='top', transform=ax000.transAxes,
           fontsize=textfontsize)
# fig.text(0.65, 0.25, r'$P(10) = {:.4f}$'.format(
#          P010['10'][-1]), fontsize=textfontsize)
fig.text(0.5, 0.1,
         r'CZ gate phase accumulation: '
         + '$\phi_{00} + \phi_{11} - \phi_{10} - \phi_{01} = $'
         + r'${:.3f} \pi $'.format(phase_accum),
         fontsize=textfontsize, ha='center')
fig.text(0.5, 0.05,
         r'Fidelity: '
         + r'$F = {:.6f}$'.format(1-infidelity(xopt.x)),
         fontsize=textfontsize, ha='center')

for axarr in axes:
    for ax in axarr:
        ax.legend(loc='lower left')
        ax.set_xlim([np.min(t_points), np.max(t_points)])
        ax.set_xlabel('Time (ns)')
        ax.set_ylim([0, 1.02])
        ax.set_ylabel(r'$P_{i\rightarrow f}$')

ax011.set_title(
    r'Starting in $|11\rangle$')
ax010.set_title(
    r'Starting in $|10\rangle$')

ax001.set_title(
    r'Starting in $|01\rangle$')
ax000.set_title(
    r'Starting in $|00\rangle$')

fig.tight_layout(rect=[0, 0.15, 1, 1])

P_driven_transition = gates.prob_transition(U_t, initial_state, final_state)

t_2nd_excited = scipy.integrate.trapz(P_driven_transition, t_points)
print('Time spent in the 2nd state for {} - {}: {:.1f} ns'.format(
    transition_to_drive[0], transition_to_drive[1],
    t_2nd_excited))

# Sweep gate time (optional)

In [ ]:
#Gate parameter
T_gate_array = np.linspace(20,100,41) #ns
error_array = np.zeros_like(T_gate_array)
drag_coeff_array = np.zeros_like(T_gate_array)
delta_omega_d_array = np.zeros_like(T_gate_array)

for T_idx, T_gate in enumerate(T_gate_array):
  
    t_points = np.linspace(0, T_gate, 10 * int(T_gate) + 1)
    delta_omega_d = 0
    drag_coeff = 0

    x0 = [delta_omega_d, drag_coeff]
    xopt = minimize(infidelity, x0, method ='Powell', tol = 1e-6)
    
    error_array[T_idx] = infidelity(xopt.x)
    delta_omega_d[T_idx] = xopt.x[0]
    drag_coeff[T_idx] = xopt.x[1]

# Pauli Transfer Matrix

In [ ]:
prep_ops = [qeye(nlev_q ), op.sx(phi=np.pi/2, N=nlev_q ), op.sy(phi=-np.pi/2, N=nlev_q ), op.sigx(N=nlev_q )]
meas_ops = [qeye(nlev_q ), op.sigx(N=nlev_q ), op.sigy(N=nlev_q ), op.sigz(N=nlev_q )]
prep_rotations = []
for q1gate in prep_ops:
    for q2gate in prep_ops:
        gate = (tensor(q1gate, q2gate))
        prep_rotations.append(gate)
meas_rotations = []
for q1gate in meas_ops:
    for q2gate in meas_ops:
        gate = (tensor(q1gate, q2gate))
        meas_rotations.append(gate)
p_in = np.zeros((16,16), dtype = complex)
p_out = np.zeros((16,16), dtype = complex)
# state_0 = tensor(basis(nlev_q,0), basis(nlev_q,0)) 
state_0 = system.eigvec('00')
for i, prep in enumerate(prep_rotations):
    psi_ini = prep*state_0
    evol_states = U_t*psi_ini
#     evol_states = gates.evolution_psi_microwave(system.H(), H_drive, psi0 = psi_ini, comp_space=comp_space, 
#                                                 t_points=t_points,T_gate=T_gate, shape=shape, 
#                                                 sigma=sigma, DRAG = drag, DRAG_coefficient = drag_coeff, 
#                                                 omega_d=omega_d,interaction=interaction)
    
    for j, meas in enumerate(meas_rotations):
        p_in[i,j] = np.real(expect(meas, psi_ini))
        p_out[i,j] = np.real(expect(meas, single_qubit_gates*evol_states[-1]))
        
ptm_real =  np.linalg.lstsq(p_in, p_out, rcond=None)[0]

str_primitive = ["I","X","Y","Z"]
op_label = []
for prim1 in str_primitive:
    for prim2 in str_primitive:
        op_label.append(prim1+prim2)
# hinton(ptm_real, xlabels = op_label, ylabels = op_label)  

In [ ]:
pm = ptm_real
choi_matrix = tomography.ptm_to_choi(pm)
choi_physical = tomography.project_and_normalize_density_matrix(choi_matrix)
pm_physical = tomography.choi_to_ptm(choi_physical)
pm = pm_physical
print (pm)

In [ ]:
hinton(ptm_real, xlabels = op_label, ylabels = op_label) 

# Estimating error channel

In [ ]:
#List of collapse operators
gamma1_01 = (10e6)**-1.0
gamma2_01 = (5e3)**-1.0
L1_01 = np.sqrt(gamma1_01)*(system.eigvec('00')*system.eigvec('01').dag()+system.eigvec('00')*system.eigvec('10').dag())
L2_01 = np.sqrt(gamma2_01/2.0)*(system.eigvec('00')*system.eigvec('00').dag() - system.eigvec('01')*system.eigvec('01').dag() - system.eigvec('10')*system.eigvec('10').dag() + system.eigvec('11')*system.eigvec('11').dag())

gamma1_02 = (50e3)**-1
gamma2_02 = (30e3)**-1
L1_02 = np.sqrt(gamma1_02)*(system.eigvec('00')*system.eigvec('02').dag() + system.eigvec('00')*system.eigvec('20').dag())
L2_02 = np.sqrt(gamma2_02/2.0)*(system.eigvec('00')*system.eigvec('00').dag()-system.eigvec('02')*system.eigvec('02').dag() - system.eigvec('20')*system.eigvec('20').dag()+system.eigvec('22')*system.eigvec('22').dag())

gamma1_12 = (240e3)**-1
gamma2_12 = (6.5e3)**-1
L1_12 = np.sqrt(gamma1_12)*(system.eigvec('10')*system.eigvec('20').dag() + system.eigvec('11')*system.eigvec('21').dag()
                           + system.eigvec('01')*system.eigvec('02').dag() + system.eigvec('11')*system.eigvec('12').dag())

L2_12 = np.sqrt(gamma2_12/2.0)*(system.eigvec('01')*system.eigvec('01').dag()-system.eigvec('02')*system.eigvec('02').dag() + system.eigvec('11')*system.eigvec('11').dag()-system.eigvec('12')*system.eigvec('12').dag()
                               + system.eigvec('10')*system.eigvec('10').dag()-system.eigvec('20')*system.eigvec('20').dag() - system.eigvec('21')*system.eigvec('21').dag() + system.eigvec('22')*system.eigvec('22').dag())


c_ops = [L1_01,L2_01, L1_02,L2_02, L1_12,L2_12]

In [ ]:
prep_ops = [qeye(nlev_q ), op.sx(phi=np.pi/2, N=nlev_q ), op.sy(phi=-np.pi/2, N=nlev_q ), op.sigx(N=nlev_q )]
meas_ops = [qeye(nlev_q ), op.sigx(N=nlev_q ), op.sigy(N=nlev_q ), op.sigz(N=nlev_q )]
prep_rotations = []
for q1gate in prep_ops:
    for q2gate in prep_ops:
        gate = (tensor(q1gate, q2gate))
        prep_rotations.append(gate)
meas_rotations = []
for q1gate in meas_ops:
    for q2gate in meas_ops:
        gate = (tensor(q1gate, q2gate))
        meas_rotations.append(gate)
p_in_diss = np.zeros((16,16), dtype = complex)
p_out_diss = np.zeros((16,16), dtype = complex)
# state_0 = tensor(basis(nlev_q,0), basis(nlev_q,0))
state_0 = system.eigvec('00')
for i, prep in enumerate(prep_rotations):
    psi_ini = prep*state_0
    evol_states = gates.evolution_psi_microwave_diss(system.H(), H_drive, psi0 = psi_ini, comp_space=comp_space, 
                                                t_points=t_points,T_gate=T_gate, shape=shape, 
                                                sigma=sigma, DRAG = drag, DRAG_coefficient = drag_coeff, 
                                                omega_d=omega_d,interaction=interaction, c_ops = c_ops)
    
    for j, meas in enumerate(meas_rotations):
        p_in_diss[i,j] = np.real(expect(meas, psi_ini))
        p_out_diss[i,j] = np.real(expect(meas, single_qubit_gates*evol_states[-1]*single_qubit_gates.dag()))
         

In [ ]:
ptm_real_diss =  np.linalg.lstsq(p_in_diss, p_out_diss, rcond=None)[0]

str_primitive = ["I","X","Y","Z"]
op_label = []
for prim1 in str_primitive:
    for prim2 in str_primitive:
        op_label.append(prim1+prim2)
# hinton(ptm_real_diss, xlabels = op_label, ylabels = op_label) 

In [ ]:
pm = ptm_real_diss
choi_matrix = tomography.ptm_to_choi(pm)
choi_physical = tomography.project_and_normalize_density_matrix(choi_matrix)
pm_physical = tomography.choi_to_ptm(choi_physical)
ptm_real_diss = pm_physical

In [ ]:
# print(ptm_real_diss)
print (tomography.process_fidelity(ptm_real_diss, ptm_ideal))

## Ideal PTM

In [ ]:
prep_generator = [qeye(2), ry(np.pi/2.0), rx(np.pi/2.0), sigmax()]
prep_rotations = []
for q1gate in prep_generator:
    for q2gate in prep_generator:
        gate = (tensor(q1gate, q2gate))
        prep_rotations.append(gate)

meas_generator = [qeye(2), sigmax(), sigmay(), sigmaz()]
meas_rotations = []
for q1gate in meas_generator:
    for q2gate in meas_generator:
        gate = (tensor(q1gate, q2gate))
        meas_rotations.append(gate)
        
p_in = np.zeros((16,16)) 
p_out = np.zeros((16,16)) 

gnd_state = tensor(basis(2,0), basis(2,0))
# gate_op = tensor(qeye(2), rx(np.pi/2))
# gate_op = tensor(rx(np.pi/2), qeye(2))
# gate_op = cnot()
gate_op = cphase(np.pi)
for i, rot_i in enumerate(prep_rotations):
    for j, rot_j in enumerate(meas_rotations):
        p_in[i,j] = expect(rot_j, rot_i*gnd_state)
        p_out[i,j] = expect(rot_j, gate_op*(rot_i*gnd_state))        
ptm_ideal = np.linalg.lstsq(p_in, p_out, rcond = None)[0]
hinton(ptm_ideal, xlabels = op_label, ylabels = op_label) 

In [ ]:
ptm_error = ptm_real_diss@np.linalg.inv(ptm_ideal)
hinton(ptm_error, xlabels = op_label, ylabels = op_label) 
print (tomography.process_fidelity(ptm_real_diss, ptm_ideal))

In [ ]:
print (ptm_error)

## PTM to $\chi$

In [ ]:
prep_generator = [qeye(2), rx(np.pi/2.0), ry(np.pi/2.0), sigmax()]
prep_rotations = []
for q1gate in prep_generator:
    for q2gate in prep_generator:
        gate = (tensor(q1gate, q2gate))
        prep_rotations.append(gate)

meas_generator = [qeye(2), sigmax(), sigmay(), sigmaz()]
meas_rotations = []
for q1gate in meas_generator:
    for q2gate in meas_generator:
        gate = (tensor(q1gate, q2gate))
        meas_rotations.append(gate)
        
p_in = np.zeros((16,16))  
gnd_state = tensor(basis(2,0), basis(2,0))
for i, rot_i in enumerate(prep_rotations):
    for j, rot_j in enumerate(meas_rotations):
        p_in[i,j] = expect(rot_j, rot_i*gnd_state)
        
# Define set of input states
rho_input = []
for i, prep in enumerate(prep_rotations):
    rho = prep*gnd_state
    rho = ket2dm(rho)
    rho_input.append(rho)
    
#Get output states from PTM    
p_out = p_in@ptm_error
# p_out = p_in@ptm_real
rho_output = []
for idx in range (len(rho_input)):
    rho = 0
    for j, meas in enumerate(meas_rotations):
        rho = rho + 0.25*p_out[idx,j]*meas
    rho_output.append(rho)
    
# Quantum process tomography
n = 2
d = 2**n
la = np.zeros(d**4, dtype = complex)
chi = np.zeros(d**4, dtype = complex)
beta = np.zeros((d**4,d**4), dtype = complex)
for j in range(16):
    for k in range(16):
        la[k+16*j] = (rho_output[j]*rho_input[k]).tr()
        for m in range(16):
            for n in range(16):
                 beta[k+16*j, n+16*m] = (meas_rotations[m]*rho_input[j]*meas_rotations[n]*rho_input[k]).tr()

kappa = np.linalg.inv(beta)
chi = kappa.dot(la)
chi = np.reshape(chi, (16,16)).transpose()
# for m in range(4):
#     for n in range(4):
#         for j in range(4):
#             for k in range(4):
#                 chi[n+4*m] = chi[n+4*m]+ kappa[m,n,j,k]*la[j,k]

op_label = [["$I$", "$X$", "$Y$", "$Z$"] for i in range (2)]

qpt_plot_combined(chi, op_label)    

In [ ]:
fname = r'/Users/longnguyen/Google Drive/LBL Research/Data/BNF simulation/BNF_CZ_error_chi.npy'
np.save(fname, chi)

In [ ]:
print (np.array(chi))

In [ ]:
for idx in range(16):
    for idy in range(16):
        if idx == idy:
            print (abs(chi[idx, idy]))

In [ ]:
print (abs(chi[0,3])) # IZ
print (abs(chi[0,12])) # ZI
print (abs(chi[0,15])) # ZZ

## PTM to operator

In [ ]:
from qiskit.quantum_info import Operator, Choi, Kraus, PTM, Chi
from qiskit import visualization
op = Operator(np.array(cphase(np.pi)))
ptm = PTM(op)

# print (ptm)
# ptm = PTM(op)
# print (np.array(ptm)-ptm_ideal_2q)
# print (ptm)
# print (ptm_ideal_2q)
# visualization.plot_state_hinton (ptm)

# cz_uni = ptm.to_operator()
chk = np.array(cz_uni)-np.array(cphase(np.pi))
print (np.round(chk,1))

## $\chi$ to unitary operator

In [ ]:
eig_val, eig_vec = Qobj(chi).eigenstates()

In [ ]:
eig_vec[-1]

In [ ]:
uni = np.zeros((4,4))

for t in range(16):
    uni = uni + np.array(meas_rotations[t]*np.conjugate(eig_vec[-1][t][0,0]))

In [ ]:
np.round(uni,2)